In [2]:
import threading
import time

def background_task():
    while True:
        print("Tache en arriere-plan")
        time.sleep(1)


# Creer un thread en arriere-plan (daemon)
background_thread = threading.Thread(target = background_task)
background_thread.daemon = True

# Demarrer le thread en arrier-plan
background_thread.start()

print("start")
time.sleep(5)
print("end")
print("Fin du programme principale")

Tache en arriere-plan
Tache en arriere-plan
start
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
end
Fin du programme principale
Tache en arriere-plan


Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan
Tache en arriere-plan


# Probleme 1
### Un agent globale qui doit collecter des informations provenant de trois locaux fonctionnant en parallele 

In [3]:
import threading

class GlobalAgent:
    
    def __init__(self):
        self.lock = threading.Lock()
        self.data = []
        
    def receive_data(self, agent_id, info):
        with self.lock:
            self.data.append(f"Agent {agent_id} : info")


def local_agent(agent_id, global_agent):
    for i in range(5):
        info = f" Data {i}"
        global_agent.receive_data(agent_id, info)
        print(f"Agent {agent_id} sent : {info}")
        

In [4]:
global_agent = GlobalAgent()

agent_1 = threading.Thread(target=local_agent, args=(1, global_agent))
agent_2 = threading.Thread(target=local_agent, args=(2, global_agent))
agent_3 = threading.Thread(target=local_agent, args=(3, global_agent))

agent_1.start()
agent_2.start()
agent_3.start()

agent_1.join()
agent_2.join()
agent_3.join()

print("Donnees collectees par l'agent globale")
for data in global_agent.data:
    print(data)

Agent 1 sent :  Data 0
Agent 1 sent :  Data 1
Agent 1 sent :  Data 2
Agent 1 sent :  Data 3
Agent 1 sent :  Data 4
Agent 2 sent :  Data 0
Agent 2 sent :  Data 1
Agent 2 sent :  Data 2
Agent 2 sent :  Data 3
Agent 2 sent :  Data 4
Agent 3 sent :  Data 0
Agent 3 sent :  Data 1
Agent 3 sent :  Data 2
Agent 3 sent :  Data 3
Agent 3 sent :  Data 4
Donnees collectees par l'agent globale
Agent 1 : info
Agent 1 : info
Agent 1 : info
Agent 1 : info
Agent 1 : info
Agent 2 : info
Agent 2 : info
Agent 2 : info
Agent 2 : info
Agent 2 : info
Agent 3 : info
Agent 3 : info
Agent 3 : info
Agent 3 : info
Agent 3 : info


## - Les agents doivent communiquer entre eux, et partager les donnees avec l'agent globale

In [8]:
import threading
import queue
import time

class GlobalAgent:
    def __init__(self, agent_count):
        self.lock = threading.Lock()
        self.data = []
        self.agents = [queue.Queue() for _ in range(agent_count)]

    def receive_data(self, agent_id, info):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        with self.lock:
            self.data.append(f"{timestamp} - Agent {agent_id}: {info}")

    def send_message(self, from_agent, to_agent, message):
        self.agents[to_agent].put(message)

def local_agent(agent_id, global_agent):
    for i in range(5):
        info = f"Data {i}"
        global_agent.receive_data(agent_id, info)
        global_agent.send_message(agent_id, (agent_id + 1) % 3, f"Message {i}")
        time.sleep(1)

if __name__ == "__main__":
    global_agent = GlobalAgent(agent_count=3)

    # Créer trois agents locaux dans leurs propres threads
    thread1 = threading.Thread(target=local_agent, args=(0, global_agent))
    thread2 = threading.Thread(target=local_agent, args=(1, global_agent))
    thread3 = threading.Thread(target=local_agent, args=(2, global_agent))

    # Démarrer les threads des agents locaux
    thread1.start()
    thread2.start()
    thread3.start()

    # Attendre que tous les threads se terminent
    thread1.join()
    thread2.join()
    thread3.join()

    # Afficher les données collectées par l'agent global
    print("Données collectées par l'agent global:")
    for data in global_agent.data:
        print(data)

    # Afficher les messages envoyés entre les agents
    print("\nMessages entre les agents:")
    for i, agent_queue in enumerate(global_agent.agents):
        while not agent_queue.empty():
            message = agent_queue.get()
            print(f"Agent {i} received: {message}")


Données collectées par l'agent global:
2023-09-30 09:03:40 - Agent 0: Data 0
2023-09-30 09:03:40 - Agent 1: Data 0
2023-09-30 09:03:40 - Agent 2: Data 0
2023-09-30 09:03:41 - Agent 0: Data 1
2023-09-30 09:03:41 - Agent 1: Data 1
2023-09-30 09:03:41 - Agent 2: Data 1
2023-09-30 09:03:42 - Agent 0: Data 2
2023-09-30 09:03:42 - Agent 1: Data 2
2023-09-30 09:03:42 - Agent 2: Data 2
2023-09-30 09:03:43 - Agent 0: Data 3
2023-09-30 09:03:43 - Agent 1: Data 3
2023-09-30 09:03:43 - Agent 2: Data 3
2023-09-30 09:03:44 - Agent 0: Data 4
2023-09-30 09:03:44 - Agent 1: Data 4
2023-09-30 09:03:44 - Agent 2: Data 4

Messages entre les agents:
Agent 0 received: Message 0
Agent 0 received: Message 1
Agent 0 received: Message 2
Agent 0 received: Message 3
Agent 0 received: Message 4
Agent 1 received: Message 0
Agent 1 received: Message 1
Agent 1 received: Message 2
Agent 1 received: Message 3
Agent 1 received: Message 4
Agent 2 received: Message 0
Agent 2 received: Message 1
Agent 2 received: Message 2

## L'agent globale doit a sont tour pouvoir communiquer avecles agents locals

In [ ]:
import threading
import queue
import time


class GlobalAgent:
    
    def __init__(self, agent_count):
        self.lock = threading.Lock()
        self.data =[]
        self.agents = [queue.Queue() for _ in range(agent_count)]
        self.global_queue = queue.Queue()
        
    def receive_data(self, agent_id, info):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        with self.lock:
            self.data.append(f"{timestamp} - Agent {agent_id} : {info}")
    
    def send_message(self, from_agent, to_agent, message):
        self.agents[to_agent].put(message)
    
    def send_global_message(self, message):
        self.global_queue.put(message)
    
    def receive_message(self, agent_id):
        return self.agents[agent_id].get()
    

def local_agent(agent_id, global_agent):
    for i in range(5):
        info = f" Data {i}"
        global_agent.receive_data(agent_id, info)
        global_agent.send_message(agent_id, (agent_id + 1) % 3, f"Message {i}")
        global_message = global_agent.receive_globa_message()
        print(f"Agent {agent_id} received glo")

In [9]:
import threading
import queue
import time

class GlobalAgent:
    def __init__(self, agent_count):
        self.lock = threading.Lock()
        self.data = []
        self.agents = [queue.Queue() for _ in range(agent_count)]
        self.global_queue = queue.Queue()

    def receive_data(self, agent_id, info):
        timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
        with self.lock:
            self.data.append(f"{timestamp} - Agent {agent_id}: {info}")

    def send_message(self, from_agent, to_agent, message):
        self.agents[to_agent].put(message)

    def send_global_message(self, message):
        self.global_queue.put(message)

    def receive_message(self, agent_id):
        return self.agents[agent_id].get()

    def receive_global_message(self):
        return self.global_queue.get()

def local_agent(agent_id, global_agent):
    for i in range(5):
        info = f"Data {i}"
        global_agent.receive_data(agent_id, info)
        global_agent.send_message(agent_id, (agent_id + 1) % 3, f"Message {i}")
        global_message = global_agent.receive_global_message()
        print(f"Agent {agent_id} received global message: {global_message}")
        time.sleep(1)

if __name__ == "__main__":
    global_agent = GlobalAgent(agent_count=3)

    # Créer trois agents locaux dans leurs propres threads
    thread1 = threading.Thread(target=local_agent, args=(0, global_agent))
    thread2 = threading.Thread(target=local_agent, args=(1, global_agent))
    thread3 = threading.Thread(target=local_agent, args=(2, global_agent))

    # Démarrer les threads des agents locaux
    thread1.start()
    thread2.start()
    thread3.start()

    # Attendre que tous les threads se terminent
    thread1.join()
    thread2.join()
    thread3.join()

    # Afficher les données collectées par l'agent global
    print("Données collectées par l'agent global:")
    for data in global_agent.data:
        print(data)


## Utilisation de la file d'attente

In [1]:
import queue
import threading
import time

def worker(q):
    while True:
        try:
            item = q.get(timeout=1)
            print(f"Traitement de {item}")
            q.task_done()
        except queue.Empty:
            print("La file d'attente est vide, fin du thread.")
            break


# Creer une file d'attente
ma_file = queue.Queue()

# Creer et demarrer deux threads de travail
agent_1 = threading.Thread(target = worker, args = (ma_file, ))
agent_2 = threading.Thread(travail = worker, args = (ma_file, ))

agent_1.start()
agent_2.start()

# Mettre des elements dans la file d'attente
ma_file.put("Element 1")
ma_file.put("Element 2")
ma_file.put("Element 3")

# Attendre que les threads terminent
agent_1.join()
agent_2.join()

print("Tous les elements ont ete traites")


## Exemple d'utilisation de la file d'attente prioritaire

In [4]:
import queue

# Creer une file d'attente prioritaire
ma_file_prioritaire = queue.PriorityQueue()

# Ajouter des elements avec des priorites
ma_file_prioritaire.put((2, "Element 2"))
ma_file_prioritaire.put((1, "Element 1"))
ma_file_prioritaire.put((3, "Element 3"))

while not ma_file_prioritaire.empty():
    priority, item = ma_file_prioritaire.get()
    print(f"Element de priorite {priority} : {item}")
    

Element de priorite 1 : Element 1
Element de priorite 2 : Element 2
Element de priorite 3 : Element 3


## Exemple d'utilisation de la file d'attente de type Lifo (Last-In, First=Out)

In [3]:
import queue

# Creer une file d'attente de type Lifo
ma_file_lifo = queue.LifoQueue()

# Ajouter des elements
ma_file_lifo.put("Element 1")
ma_file_lifo.put("Element 2")
ma_file_lifo.put("Element 3")

# Recuperer les elements dans l'ordre Lifo (Last-In, First-Out)
while not ma_file_lifo.empty():
    item = ma_file_lifo.get()
    print(f" Element : {item}")
    

 Element : Element 3
 Element : Element 2
 Element : Element 1
